In [15]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np

#open a random image and check its size
im = Image.open("/Users/stella/Downloads/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg")
#display.display(Image.open(str('/Users/stella/Downloads/Animals_with_Attributes2_3/JPEGImages/hippopotamus/hippopotamus_10003.jpg')))
width, height = im.size 
print(width,height)

#resizing
im=im.resize((448,448),resample=0)
#im.show()




320 223


In [16]:
#11788 images in this dataset

In [17]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, '/')
  # The second to last is the class-directory
  return parts[-2] 

[IMG_WIDTH, IMG_HEIGHT]=[448,448]
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [18]:
list_ds = tf.data.Dataset.list_files(str('/Users/stella/Downloads/CUB_200_2011/CUB_200_2011/images/*/*'))
labeled_ds = list_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)
type(labeled_ds)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [19]:
labeled_ds

<DatasetV1Adapter shapes: ((448, 448, 3), ()), types: (tf.float32, tf.string)>

In [20]:
# source: https://www.tensorflow.org/tutorials/load_data/images
#randomly take one image and see the image and its label
for image, label in labeled_ds.take(1):
  print("Image shape: ", image.numpy())
  print("Label: ", label)

Image shape:  [[[0.38107497 0.37254903 0.3923845 ]
  [0.3942052  0.3732318  0.39812678]
  [0.40505955 0.37988448 0.4089811 ]
  ...
  [0.77647066 0.7568628  0.7803922 ]
  [0.77647066 0.7568628  0.7803922 ]
  [0.7727766  0.76425093 0.7840863 ]]

 [[0.3945503  0.37800246 0.40056464]
  [0.40463734 0.37916    0.4058322 ]
  [0.41244847 0.3872734  0.41443455]
  ...
  [0.7682905  0.7568628  0.7776655 ]
  [0.7705425  0.7591148  0.7799175 ]
  [0.7724602  0.7644091  0.7840863 ]]

 [[0.4291834  0.39969367 0.42719895]
  [0.43329248 0.40005106 0.42976823]
  [0.43330753 0.4029021  0.43251532]
  ...
  [0.76221573 0.7563028  0.7759106 ]
  [0.7644205  0.75850755 0.7781154 ]
  [0.7684613  0.7625483  0.78215617]]

 ...

 [[0.18374653 0.10207281 0.03143382]
  [0.17215607 0.10423766 0.03142316]
  [0.16198355 0.1081405  0.03307059]
  ...
  [0.8126183  0.78516734 0.8126183 ]
  [0.8267115  0.78954405 0.8202339 ]
  [0.82574844 0.79761493 0.8252934 ]]

 [[0.18624826 0.10209949 0.03655832]
  [0.17702001 0.1048252

In [21]:
#SPLIT TRAIN,VAL,TEST
DATASET_SIZE=37322
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

full_dataset = labeled_ds
raw_train = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
raw_validation = test_dataset.skip(test_size)
raw_test = test_dataset.take(test_size)

In [22]:
print(raw_train)
print(raw_validation)
print(raw_test)

<DatasetV1Adapter shapes: ((448, 448, 3), ()), types: (tf.float32, tf.string)>
<DatasetV1Adapter shapes: ((448, 448, 3), ()), types: (tf.float32, tf.string)>
<DatasetV1Adapter shapes: ((448, 448, 3), ()), types: (tf.float32, tf.string)>


In [23]:


def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label


In [24]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [25]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [26]:
for image_batch, label_batch in train_batches.take(1):
   pass

image_batch.shape

TensorShape([32, 448, 448, 3])

In [27]:
base_model = tf.keras.applications.vgg19.VGG19(include_top=False, 
                                               weights='imagenet', 
                                               input_tensor=None, 
                                               input_shape=None,
                                               pooling=None, 
                                               classes=1000)

In [28]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 14, 14, 512)


In [29]:
#FREEZING

base_model.trainable = False

